In [2]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import f1_score,precision_score
from PIL import Image
from torchvision.transforms import transforms 
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import torch.optim as optim
from torch.optim import lr_scheduler
from pathlib import Path
import math
import copy

In [3]:
image_path='./img_align_celeba'
df_attr=pd.read_csv('./list_attr_celeba.csv')
df_attr.replace(-1,0,inplace=True)


In [4]:
df = pd.DataFrame()
df['image_id'] = df_attr['image_id']
df['Eyeglasses'] = df_attr['Eyeglasses']
df_attr = df
df_attr.head()

,image_id,Eyeglasses
0,000001.jpg,0
1,000002.jpg,0
2,000003.jpg,0
3,000004.jpg,0
4,000005.jpg,0


In [13]:
# df_attr[df_attr['image_id'] == '000579.jpg']
df_attr[df_attr['Eyeglasses'] == 1.0]

,image_id,Eyeglasses
52,000053.jpg,1
92,000093.jpg,1
118,000119.jpg,1
143,000144.jpg,1
151,000152.jpg,1
...,...,...
202540,202541.jpg,1
202567,202568.jpg,1
202587,202588.jpg,1
202589,202590.jpg,1


In [16]:
df_attr.iloc[52]

image_id      000053.jpg
Eyeglasses             1
Name: 52, dtype: object

In [7]:
attr=df_attr.drop(['image_id'],axis=1)

In [17]:
np.asarray(attr.iloc[52].T,dtype=np.float32)

array([1.], dtype=float32)

In [4]:
# Class to get data in specific format and preprocessing.
class CelebDataset(Dataset):
    def __init__(self,df_1,image_path,transform=None,mode='train'):
        super().__init__()
        self.attr=df_1['Eyeglasses']
        self.path=image_path
        self.image_id=df_1['image_id']
        self.transform=transform
        self.mode=mode
    def __len__(self):
        return self.image_id.shape[0]
    def __getitem__(self,idx:int):
        image_name=self.image_id.iloc[idx]
        image=Image.open(os.path.join(image_path,image_name))
        attributes=np.asarray(self.attr.iloc[idx].T,dtype=np.float32)
        labels=self.attr.columns.tolist()
        if self.transform:
            image=self.transform(image)
        return image,attributes    

In [5]:
# Split Dataset into train,test and valid
from sklearn.model_selection import train_test_split
train_df,test=train_test_split(df_attr,test_size=0.1,shuffle=True,random_state=212)
valid_df,test_df=train_test_split(test,test_size=0.5,random_state=212)
train_df.shape,valid_df.shape,test_df.shape

((182339, 2), (10130, 2), (10130, 2))

In [6]:
# Apply Data augmentation and different type of transforms on train data.
train_transform=transforms.Compose([transforms.Resize((224,224)),transforms.RandomVerticalFlip(p=0.5),
                                    transforms.RandomHorizontalFlip(p=0.5),transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.5063, 0.4258, 0.3832],std=[0.2644, 0.2436, 0.2397])])
# Apply Data augmentation and different type of transforms on test and validation data.
valid_transform=transforms.Compose([transforms.Resize((224,224)),
                              transforms.ToTensor(),
                              transforms.Normalize(mean=[0.5063, 0.4258, 0.3832],std=[0.2644, 0.2436, 0.2397])])

In [7]:
# Create Dataset Object and DataLoader for train and validation set 
train_data=CelebDataset(train_df,image_path,train_transform)
train_loader=DataLoader(train_data,batch_size=64,shuffle=True,num_workers=4)
valid_data=CelebDataset(valid_df,image_path,valid_transform)
valid_loader=DataLoader(valid_data,batch_size=64,num_workers=4)

In [10]:
for images,attr in train_loader:
    print(attr)

BrokenPipeError: [Errno 32] Broken pipe